In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [2]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [3]:
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()

In [42]:
#dataframe for no. of cases and death 
state=pd.read_csv("covid19_jhu_csse_summary_BigQuery.csv")
state=state.groupby("province_state").max()
state=state[['confirmed', 'deaths', 'recovered', 'active']]
state.head()

,confirmed,deaths,recovered,active
province_state,,,,
"Norfolk County, MA",2,0.0,0.0,NaN
Alabama,1474,90.0,0.0,1384.0
"Alameda County, CA",2,0.0,0.0,NaN
Alaska,194,5.0,0.0,190.0
American Samoa,0,0.0,0.0,0.0


In [7]:
#dataframe for population of the state  
population=pd.read_excel("statepopulation.xlsx")
new_header = population.iloc[0] #grab the first row for the header
population = population[1:] #take the data less the header row
population.columns = new_header
def name(stat):
    try:
        return stat[1:]
    except:
        return "NA"
population["States"]=population["States"].apply(name)
population.set_index('States',inplace=True)
population.head()

,Population
States,
Alabama,4903185
Alaska,731545
Arizona,7278717
Arkansas,3017804
California,39512223


In [8]:
#dataframe for disability by state
disability=pd.read_csv("Disability_by_Type,_2014-2018.csv")
disability=disability.groupby("State").sum()
disability=disability[['\xa0Population under 18 years','\xa0Population under 18 years with one type of disability','\xa0Population under 18 years with two or more types of disability','\xa0Population 18 to 64 years','\xa0Population 18 to 64 years with one type of disability','\xa0Population 18 to 64 years with two or more types of disability','\xa0Population 65 years and over','\xa0Population 65 years and over with one type of disability','\xa0Population 65 years and over with two or more types of disability','\xa0Population with a hearing difficulty','\xa0Population with a vision difficulty', '\xa0Population 5 years and over with a cognitive difficulty', '\xa0Population 5 years and over with an ambulatory difficulty', '\xa0Population 5 years and over with a self-care difficulty','\xa0Population 18 years and over with an independent living difficulty', '\xa0Population with a disability']]
disability["population under 18 with disability"]=disability["\xa0Population under 18 years with one type of disability"]+disability['\xa0Population under 18 years with two or more types of disability']
disability["population between 18 to 64 with disability"]=disability["\xa0Population 18 to 64 years with one type of disability"]+disability["\xa0Population 18 to 64 years with two or more types of disability"]
disability["population above 64 with disability"]=disability["\xa0Population 65 years and over with one type of disability"]+disability["\xa0Population 65 years and over with two or more types of disability"]
disability.drop(["\xa0Population under 18 years with one type of disability",'\xa0Population under 18 years with two or more types of disability',"\xa0Population 18 to 64 years with one type of disability","\xa0Population 18 to 64 years with two or more types of disability","\xa0Population 65 years and over with one type of disability","\xa0Population 65 years and over with two or more types of disability"],axis=1,inplace=True)
disability.head()

,Population under 18 years,Population 18 to 64 years,Population 65 years and over,Population with a hearing difficulty,Population with a vision difficulty,Population 5 years and over with a cognitive difficulty,Population 5 years and over with an ambulatory difficulty,Population 5 years and over with a self-care difficulty,Population 18 years and over with an independent living difficulty,Population with a disability,population under 18 with disability,population between 18 to 64 with disability,population above 64 with disability
State,,,,,,,,,,,,,
Alabama,1097072,2924082,763886,203134,145490,295224,445331,155154,283390,780618,51993,416469,312156
Alaska,185685,451328,77093,32428,17152,30196,38460,14588,25072,85895,6728,49019,30148
Arizona,1631619,4059983,1146668,279497,169657,318457,453483,160957,292514,885094,65660,425886,393548
Arkansas,703847,1761927,471361,144985,101641,189628,278592,97807,172398,505899,40025,268473,197401
California,9059370,24374529,5220049,1140654,772344,1565088,2127363,951653,1621861,4089685,295092,1971981,1822612


In [21]:
#dataframe for number of beds
bed=pd.read_csv("Definitive_Healthcare__USA_Hospital_Beds.csv")
bed=bed.groupby("STATE_NAME").sum()
bed=bed["NUM_LICENSED_BEDS"]
bed=bed.to_frame()
bed.head()

,NUM_LICENSED_BEDS
STATE_NAME,
Alabama,18611.0
Alaska,1832.0
Arizona,18345.0
Arkansas,12729.0
California,83764.0


In [22]:
#dataframe for Number of practitoner required 
pract=pd.read_excel("health_professional_shortage_src-bigquery.xlsx")
pract=pract.groupby("State_Name").sum()
pract=pract["HPSA_Shortage"]
pract=pract.to_frame()
pract.head()

,HPSA_Shortage
State_Name,
Alabama,4282.7776
Alaska,64.1000
American Samoa,86.7000
Arizona,18449.4136
Arkansas,650.4779


In [55]:
#dataframe for code
code=pd.read_csv("codename.csv")
code.drop(["Unnamed: 0"],axis=1,inplace=True)
code.set_index('state',inplace=True)
code.head()

,code
state,
Alabama,AL
Alaska,AK
Arizona,AZ
Arkansas,AR
California,CA


In [140]:
#Joining all the dataframes together
final = pd.concat([code,pract,bed,population,state,disability], axis=1, join='inner')

In [141]:
final.head()

,code,HPSA_Shortage,NUM_LICENSED_BEDS,Population,confirmed,deaths,recovered,active,Population under 18 years,Population 18 to 64 years,...,Population with a hearing difficulty,Population with a vision difficulty,Population 5 years and over with a cognitive difficulty,Population 5 years and over with an ambulatory difficulty,Population 5 years and over with a self-care difficulty,Population 18 years and over with an independent living difficulty,Population with a disability,population under 18 with disability,population between 18 to 64 with disability,population above 64 with disability
Alabama,AL,4282.7776,18611.0,4903185,1474,90.0,0.0,1384.0,1097072,2924082,...,203134,145490,295224,445331,155154,283390,780618,51993,416469,312156
Alaska,AK,64.1000,1832.0,731545,194,5.0,0.0,190.0,185685,451328,...,32428,17152,30196,38460,14588,25072,85895,6728,49019,30148
Arizona,AZ,18449.4136,18345.0,7278717,5988,250.0,1.0,5738.0,1631619,4059983,...,279497,169657,318457,453483,160957,292514,885094,65660,425886,393548
Arkansas,AR,650.4779,12729.0,3017804,941,23.0,0.0,934.0,703847,1761927,...,144985,101641,189628,278592,97807,172398,505899,40025,268473,197401
Colorado,CO,3670.9475,13929.0,5758736,4118,212.0,0.0,3906.0,1257857,3455987,...,190838,105888,208623,268091,93421,179721,575430,41699,300316,233415


In [142]:
final.columns

Index(['code', 'HPSA_Shortage', 'NUM_LICENSED_BEDS', 'Population', 'confirmed',
       'deaths', 'recovered', 'active', ' Population under 18 years',
       ' Population 18 to 64 years', ' Population 65 years and over',
       ' Population with a hearing difficulty',
       ' Population with a vision difficulty',
       ' Population 5 years and over with a cognitive difficulty',
       ' Population 5 years and over with an ambulatory difficulty',
       ' Population 5 years and over with a self-care difficulty',
       ' Population 18 years and over with an independent living difficulty',
       ' Population with a disability', 'population under 18 with disability',
       'population between 18 to 64 with disability',
       'population above 64 with disability'],
      dtype='object')

In [143]:
final["no of corona affected per thousand"]=(final["confirmed"]/final["Population"])*1000
final["no of corona death per thousand"]=(final["deaths"]/final["Population"])*1000
final["no of disable people per thousand"]=(final['\xa0Population with a disability']/final["Population"])*1000
final["no of hospital bed per thousand"]=(final["NUM_LICENSED_BEDS"]/final["Population"])*1000
final["no of corona death per thousand affacted"]=(final["deaths"]/final["confirmed"])*1000

In [144]:
print(final.columns)

Index(['code', 'HPSA_Shortage', 'NUM_LICENSED_BEDS', 'Population', 'confirmed',
       'deaths', 'recovered', 'active', ' Population under 18 years',
       ' Population 18 to 64 years', ' Population 65 years and over',
       ' Population with a hearing difficulty',
       ' Population with a vision difficulty',
       ' Population 5 years and over with a cognitive difficulty',
       ' Population 5 years and over with an ambulatory difficulty',
       ' Population 5 years and over with a self-care difficulty',
       ' Population 18 years and over with an independent living difficulty',
       ' Population with a disability', 'population under 18 with disability',
       'population between 18 to 64 with disability',
       'population above 64 with disability',
       'no of corona affected per thousand', 'no of corona death per thousand',
       'no of disable people per thousand', 'no of hospital bed per thousand',
       'no of corona death per thousand affacted'],
      dtype='o

In [145]:
import plotly.plotly as py
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [146]:
init_notebook_mode(connected=True) 

In [147]:
data = dict(type='choropleth',
            colorscale = 'YlOrRd',
            locations = final["code"],
            z = final['no of corona affected per thousand'],
            locationmode = 'USA-states',
            text = final['confirmed'],
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
            colorbar = {'title':"Number of people"}
            ) 
layout = dict(title = 'Number of People affected by corona per thousand people',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)

In [148]:
data = dict(type='choropleth',
            colorscale = 'YlOrRd',
            locations = final["code"],
            z = final['no of corona death per thousand'],
            locationmode = 'USA-states',
            text = final['deaths'],
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
            colorbar = {'title':"Number of people"}
            ) 
layout = dict(title = 'Number of People Died due to corona per thousand people-Mortality Rate',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)

In [149]:
data = dict(type='choropleth',
            colorscale = 'YlOrRd',
            locations = final["code"],
            z = final['no of hospital bed per thousand'],
            locationmode = 'USA-states',
            text = final['NUM_LICENSED_BEDS'],
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
            colorbar = {'title':"Number of people"}
            ) 
layout = dict(title = 'Number of Licensed Hospital Beds available per thousand people',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)

In [150]:
data = dict(type='choropleth',
            colorscale = 'YlOrRd',
            locations = final["code"],
            z = final['no of corona death per thousand affacted'],
            locationmode = 'USA-states',
            text = final['deaths'],
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
            colorbar = {'title':"Number of people"}
            ) 
layout = dict(title = 'Number of people died per thousand people affected-FATALITY RATE',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)

In [151]:
data = dict(type='choropleth',
            colorscale = 'YlOrRd',
            locations = final["code"],
            z = final['no of disable people per thousand'],
            locationmode = 'USA-states',
            text = final["\xa0Population with a disability"],
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
            colorbar = {'title':"Number of people"}
            ) 
layout = dict(title = 'Number of Disabled people per thousand people',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)

In [152]:
'''
HPSA_Shortage is
Number of full-time equivalent (FTE) practitioners needed in the Health Professional Shortage Area (HPSA) 
so that it will achieve the population to practitioner target ratio. 
The target ratio is determined by the type (discipline) of the HPSA.
'''
data = dict(type='choropleth',
            colorscale = 'YlOrRd',
            locations = final["code"],
            z = final['HPSA_Shortage'],
            locationmode = 'USA-states',
            text = final['HPSA_Shortage'],
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
            colorbar = {'title':"Number of people"}
            ) 
layout = dict(title = 'HPSA_Shortage--No. of full-time equivalent(FTE) practitioners needed in the Health Professional Shortage AREA',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)